# url: https://www.youtube.com/watch?v=HD2klICg-bA&t=1109s

### 遺伝的アルゴリズムとは
データを一定のルールに従って変化させていき、試行錯誤で最適な解を求める手法。

（巡回セールスマン問題、など）

### ソルバーによる最適化との違い
- ソルバー...最適解を**厳密**に求めようとする
- 遺伝的アルゴリズム...最適解を**探索的**に求める(メタヒューリスティクス)

### 例題

0か1を取る100個の数列がある。

数列の偶数番目の合計から、奇数番目の合計を引いた時の値が最大となる数列は何か？


In [1]:
!pip install deap

In [2]:
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
import random
import numpy as np

#最小化 or 最大化のどちらを求めたいか
creator.create("FitnessMax", base.Fitness, weights=(1.0,)) #weights=(1.0,)最大化したいときは正数(1.0), 最小化したい時は負数(-1.0,)
# 最大化したい変数が一つしかないならweights=(1.0,)でカンマの後の部分は空白にする

#個体の定義
creator.create("Individual", list, fitness=creator.FitnessMax) #list型で個体を定義する

toolbox = base.Toolbox()
#各個体の遺伝子の中身を決める関数(0 or 1)
toolbox.register("attr_bool", random.randint, 0, 1)
#個体に含まれる遺伝子の数
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=100)

#集団の個体数を設定するための関数
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
#次世代に子を残す親を選択する方法
toolbox.register("select", tools.selTournament, tournsize=5)
#交叉関数
toolbox.register("mate", tools.cxTwoPoint)
#突然変異関数（indpb=遺伝子が突然変異を起こす確率)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)

#最適化させるのは何？
def eval_sample(individual):
    ans = sum(individual[1::2]) - sum(individual[::2])
    return ans,

toolbox.register("evaluate", eval_sample)

###アルゴリズムのパラメータ設定
random.seed(64) #乱数を固定
NGEN = 20 #何世代まで行うか
POP = 10 #集団の個体数
CXPB = 0.8 #交叉確率
MUTPB = 0.1 #個体が突然変異を起こす確率

#集団情報の設定
pop = toolbox.population(n=POP)

#集団内の個体それぞれの目的関数を計算
for individual in pop:
    individual.fitness.values = toolbox.evaluate(individual)

#良い結果の個体を抽出
hof = tools.ParetoFront()

#シンプルな手法を適用
algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, halloffame=hof)

#最終的な集団からベストな個体を選ぶ
best_ind = tools.selBest(pop, 1)[0]

#結果表示
print(best_ind)
print(best_ind.fitness.values)

# nevals: この世代で、新たに評価が行われた個体の数

gen	nevals
0  	0     
1  	9     
2  	10    
3  	8     
4  	8     
5  	7     
6  	10    
7  	7     
8  	6     
9  	10    
10 	8     
11 	9     
12 	10    
13 	6     
14 	10    
15 	6     
16 	8     
17 	10    
18 	10    
19 	7     
20 	6     
[1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
(15.0,)


In [3]:
individual = [1,2,3,4,5,6]
individual[1::2]

[1, 0, 0, 1, 1]
(14.0,)

(14.0,)

### 例題

300円で遠足に持っていくお菓子を選びたい。

同じお菓子は2つ以上買わないと決めた時、もっとも満足度の高い組み合わせは何か？

|お菓子|満足度|値段|
|---|---|---|
|チョコレート|4|80円|
|ポテトチップス|5|120円|
|クッキー|6|140円|
|グミ|3|60円|
|バナナ|7|150円|

In [4]:
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
import random
import numpy as np

#最小化？最大化？
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

#個体の定義
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
#各個体の遺伝子の中身を決める関数(0 or 1)
toolbox.register("attr_bool", random.randint, 0, 1)
#個体に含まれる遺伝子の数
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=5)

#集団の個体数を設定するための関数
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
#次世代に子を残す親を選択する方法
toolbox.register("select", tools.selTournament, tournsize=5)
#交叉関数
toolbox.register("mate", tools.cxTwoPoint)
#突然変異関数（indpb=遺伝子が突然変異を起こす確率)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)

#最適化させるのは何？
def eval_sample(individual):
    ans = np.dot(individual,[4,5,6,3,7])
    return ans,

def feasible(individual):
    if np.dot(individual,[80,120,140,60,150]) <= 300:
        return True
    else:
        return False

toolbox.register("evaluate", eval_sample)
toolbox.decorate("evaluate", tools.DeltaPenalty(feasible,-1.0))

###アルゴリズムのパラメータ設定
random.seed(64) #乱数を固定
NGEN = 100 #何世代まで行うか
POP = 50 #集団の個体数
CXPB = 0.8 #交叉確率
MUTPB = 0.1 #個体が突然変異を起こす確率

#集団情報の設定
pop = toolbox.population(n=POP)

#集団内の個体それぞれの目的関数を計算
for individual in pop:
    individual.fitness.values = toolbox.evaluate(individual)

#良い結果の個体を抽出
hof = tools.ParetoFront()

#シンプルな手法を適用
algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, halloffame=hof)

#最終的な集団からベストな個体を選ぶ
best_ind = tools.selBest(pop, 1)[0]

#結果表示
print(best_ind)
print(best_ind.fitness.values)


gen	nevals
0  	0     
1  	43    
2  	43    
3  	36    
4  	41    
5  	40    
6  	42    
7  	48    
8  	37    
9  	46    
10 	41    
11 	45    
12 	44    
13 	42    
14 	44    
15 	45    
16 	44    
17 	33    
18 	48    
19 	44    
20 	48    
21 	47    
22 	42    
23 	40    
24 	44    
25 	36    
26 	41    
27 	46    
28 	45    
29 	37    
30 	39    


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


31 	36    
32 	44    
33 	46    
34 	41    
35 	42    
36 	41    
37 	42    
38 	46    
39 	40    
40 	40    
41 	36    
42 	36    
43 	43    
44 	39    
45 	32    
46 	41    
47 	38    
48 	42    
49 	42    
50 	45    
51 	39    
52 	42    
53 	48    
54 	40    
55 	48    
56 	41    
57 	43    
58 	44    
59 	37    
60 	45    
61 	43    
62 	39    
63 	42    
64 	41    
65 	41    
66 	40    
67 	39    
68 	32    
69 	44    
70 	38    
71 	37    
72 	38    
73 	39    
74 	42    
75 	33    
76 	44    
77 	41    
78 	39    
79 	37    
80 	44    
81 	49    
82 	36    
83 	40    
84 	41    
85 	40    
86 	45    
87 	41    
88 	46    
89 	48    
90 	40    
91 	32    
92 	39    
93 	40    
94 	45    
95 	43    
96 	44    
97 	38    
98 	45    
99 	37    
100	44    
[1, 0, 0, 1, 1]
(14.0,)
